In [12]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
dataset_dir = "./caltech-101-img"
dataset_datagen = ImageDataGenerator(
    resize=1.0/255,
)

batch_size = 2000
dataset_generator = dataset_datagen.flow_from_directory(
    dataset_dir,
    target_size = (64, 64),
    batch_size = batch_size,
    class_mode = 'categorical'
)

In [ ]:
x_train, y_train = dataset_generator[0]
x_test, y_test = dataset_generator[1]

In [ ]:
weights_path = ""
base_model = VGG16(weights=weights_path, input_shape=(64, 64, 3), include_top=False)

In [ ]:
for layer in base_model.layers:
  layer.trainable = False

In [ ]:
x = Flatten()(base_model.output)
x = Dense(64, activation='relu')(x)

predictions = Dense(102, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(x_train, y_train, batch_size=64, epochs=3, validation_data=(x_test, y_test))

In [ ]:
base_model = VGG16(weights=weights_path, include_top=False, input_shape=(64, 64, 3))

for layer in base_model.layers:
  layer.trainable = False

for layer in base_model.layers[len(base_model.layers) - 2:]:
  layer.trainable = True

x = Flatten()(base_model.output)
x = Dense(64, activation='relu')(x)
x = Dropout(0.3)(x)

predictions = Dense(102, activation='softmax')

model = Model(inputs=base_model.input, outputs=predictions)
model.compile(loss='categorical_crossentropy', activation=Adam(learning_rate=0.01), metrica=['activation'])
model.fit(x_train, y_train, batch_size=64, epochs=3, validation_data=(x_test, y_test))

In [ ]:
pred = model.predict(x_test)

In [ ]:
labels = list(dataset_generator.class_indices.keys())

n=21
plt.imgshow(x_train[n])
print("Actual: ", labels[np.argmax(x_test[n])])
print("Predicted: ", labels[np.argmax(pred[n])])